In [ ]:
import json
with open('/root/Logical-Fallacies/data/dpo/arguments/dpo_with_fallacies_test.json', 'r') as f:
    test = json.load(f)

with open('/root/Logical-Fallacies/data/dpo/arguments/train.json', 'r') as f:
    train = json.load(f)

new_test_data = []
train_topics = [entry['prompt'].split('topic: ')[-1].strip() for entry in train]
found = 0
for entry in test:
    topic = entry['prompt'].split('topic: ')[-1].strip()
    if topic in train_topics:
        found+=1
        continue 
    new_test_data.append(entry)


In [ ]:
# 4,10, 11,13

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EvalPrediction
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
import pandas as pd
import numpy as np

class Model():
    def __init__(self, model_name, num_labels=14) -> None:
        self.model_name = model_name
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, ignore_mismatched_sizes=True)
        if torch.cuda.is_available():
            self.model = self.model.to('cuda')
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
    def encode(self, data, padding=True, truncation=True, return_tensors='pt'):
        texts = data['text'].tolist()
        return self.tokenizer(texts, padding=padding, truncation=truncation, return_tensors=return_tensors)
    
    def data_tensor(self, data):
        encoded = self.encode(data)
        return torch.tensor(encoded.input_ids), torch.tensor(encoded.attention_mask), torch.tensor(data['label'])
    
    def get_model(self):
        return self.model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from tqdm import tqdm
import torch



def convert_data(data, split='train'):
    num_chosen = 0
    dataset = []
    chosen_samples = set()
    for entry in data:
        instruction = entry['prompt']
        chosen = entry['chosen']
        rejected = entry['rejected']
        if chosen not in chosen_samples:
            num_chosen += 1
            chosen_samples.add(chosen)
            if num_chosen <= 500 and split=='train':
                dataset.append({'text': instruction  + ' ' + chosen, 'label': 0})
            elif split =='test' and num_chosen <= 120:
                dataset.append({'text': chosen, 'label': 0})
                
        dataset.append({'text': instruction + ' ' + rejected, 'label': entry['fallacy_type']})
    return dataset

In [ ]:
model_name = 'howey/electra-large-mnli'
model = Model(model_name, num_labels=14)

In [ ]:
data = load_dataset('json', data_files='/root/Logical-Fallacies/data/dpo/arguments/train.json', split='train')
data = pd.DataFrame(data)
def downsample_data(data, split='train'):
    training_data = []
    topics = []
    num_chosen = 0
    for i, entry in data.iterrows():
        topic=entry.prompt.split('topic: ')[-1].strip()
        if topic in topics:
            continue
        topics.append(topic)        
        
        chosen = entry.chosen
        num_chosen += 1
        rejected = entry.rejected
        if num_chosen <= 150 and split=='train':
            training_data.append({'text': entry.prompt + ' ' + chosen, 'label':0})
        elif split == 'test' and num_chosen <= 25:
            training_data.append({'text': chosen, 'label': 0})
        training_data.append({'text': entry.prompt + ', ' + rejected, 'label': entry.fallacy_type})
    return (pd.DataFrame(training_data))

from datasets import Dataset
import json
data = downsample_data(data, 'train')
with open('/root/Logical-Fallacies/data/dpo/arguments/dpo_with_fallacies_test.json') as f:
    test_dataset = json.load(f)
test_dataset = downsample_data(pd.DataFrame(test_dataset), 'test')

In [ ]:
train
len(new_test_data)

In [ ]:
new_new_test_data = []
for entry in new_test_data:
    new_new_test_data.append({'text': entry['prompt'] + ' ' + entry['chosen'], 'label': 0})
    new_new_test_data.append({'text': entry['prompt'] + ' ' + entry['rejected'], 'label': entry['fallacy_type']})

new_train_data = []
for entry in train:
    new_train_data.append({'text': entry['prompt'] + ' ' + entry['chosen'], 'label': 0})
    new_train_data.append({'text': entry['prompt'] + ' ' + entry['rejected'], 'label': entry['fallacy_type']})

train_data = pd.DataFrame(new_new_test_data)
test_data = pd.DataFrame(new_train_data)
dev_data = test_data

train_input_ids, train_attention_mask, train_labels_tensor = model.data_tensor(train_data)
dev_input_ids, dev_attention_mask, dev_labels_tensor = model.data_tensor(dev_data)
test_input_ids, test_attention_mask, test_labels_tensor = model.data_tensor(test_data)



In [ ]:


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='steps',
    eval_steps=100,
    logging_dir='./logs',
    logging_steps=10,
    logging_strategy='steps',
    save_total_limit=1,
)
from collections import Counter
def compute_metrics(pred):
    predictions, labels = pred.predictions, pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    a = []
    b = []
    misclassified = []
    for i in range(predictions.shape[0]):
       if predictions[i] != labels[i]:
           misclassified.append(predictions[i])

    print(Counter(misclassified))
 
    print('----'*5)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='micro')
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        }


In [ ]:



trainer = Trainer(
        model=model.get_model(),
        args=training_args,
        compute_metrics=compute_metrics,
        data_collator=lambda data: {
            'input_ids': torch.stack([item[0] for item in data]),
            'attention_mask': torch.stack([item[1] for item in data]),
            'labels': torch.stack([item[2] for item in data])
        },
        train_dataset=torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels_tensor),
        eval_dataset=torch.utils.data.TensorDataset(dev_input_ids, dev_attention_mask, dev_labels_tensor),
        )    
   

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('/root/Logical-Fallacies/models/fallacy_clf_trainedontest')
model.tokenizer.save_pretrained('/root/Logical-Fallacies/models/fallacy_clf_trainedontest')



In [ ]:
# modell=AutoModelForSequenceClassification.from_pretrained('/root/Logical-Fallacies/models/fallacy_clf_withoutprompt')
# tokenizer=AutoTokenizer.from_pretrained('/root/Logical-Fallacies/models/fallacy_clf_withoutprompt')

def predict(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = modell(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probs.detach().numpy()

def predict_fallacy(prompt, argument):
    if prompt: 
        input_text = prompt + " " + argument
    else:
        input_text = argument
    probs = predict(input_text)

    predicted_label = np.argmax(probs)
    # for key, value in CLASSES.items():
    #     if value == predicted_label:
    #         predicted_label = key
    return predicted_label


with open('/root/Logical-Fallacies/data/dpo/arguments/dpo_with_fallacies_test.json', 'r') as f:
        test = json.load(f)



In [ ]:
with open('/root/Logical-Fallacies/results/llama/args.json', 'r') as f:
        args = json.load(f)

args = [x[1] for x in args]



In [ ]:
def chosen_accuracy(arr):
     return sum([1 for i in arr if i == 0])/len(arr)
def rejected_accuracy(arr):
     pred = [i[0] for i in arr]
     true = [i[1] for i in arr]
     return sum([1 for i in range(len(pred)) if pred[i] == true[i]])/len(pred)
preds_chosen = []
preds_rejected = []
for i, sample in tqdm(enumerate(args), total=len(args)):
     if i%50 == 0 and i >0:
          print("preds chosen")
          print(chosen_accuracy(preds_chosen))
          print("preds rejected")
          print(rejected_accuracy(preds_rejected))
          print("----"*10)
     preds_chosen.append(predict_fallacy(sample['prompt'], sample['chosen']))
     
     preds_rejected.append((predict_fallacy(sample['prompt'], sample['rejected']), sample['fallacy_type']))
     #print(predict_fallacy(sample['prompt'], sample['rejected']))
     

In [ ]:
preds_rejected

import json

In [3]:
import json
with open('/root/Logical-Fallacies/data/dpo/arguments/train.json', 'r') as f:
    df_train = json.load(f)

with open('/root/Logical-Fallacies/data/dpo/arguments/dpo_with_fallacies_test.json', 'r') as f:
    df_test = json.load(f)

train_data = []
test_data = []
limit = 430 
def downsample_data(data, split='train'):
    training_data = []
    chosen_samples = set()
    num_chosen = 0
    for i, entry in data.iterrows():
        chosen = entry.chosen
        num_chosen += 1
        rejected = entry.rejected
    
        
        if chosen not in chosen_samples:
            chosen_samples.add(chosen)
            if num_chosen <= 2000 and split=='train':
                training_data.append({'text': chosen, 'label': INVERSE[0]})
            elif split == 'test' and num_chosen <= 500:
                training_data.append({'text': chosen, 'label': INVERSE[0]})


        training_data.append({'text': rejected, 'label': INVERSE[entry['fallacy_type']]})
    return (pd.DataFrame(training_data))

CLASSES = {'Not a Fallacy': 0,  'faulty generalization': 1, 'false causality': 2, 'fallacy of relevance': 3, 'fallacy of extension': 4, 'equivocation': 5, 'ad populum': 6, 'appeal to emotion': 7, 'ad hominem': 8, 'circular reasoning': 9, 'fallacy of credibility': 10, 'fallacy of logic': 11, 'false dilemma': 12, 'intentional': 13}
INVERSE = {v:k for k,v in CLASSES.items()}


# for entry in df_train:
#     train_data.append({'text': entry['chosen'], 'label': INVERSE[0]})
#     train_data.append({'text': entry['rejected'], 'label': INVERSE[entry['fallacy_type']]})

# for entry in df_test:
#     test_data.append({'text': entry['chosen'], 'label': INVERSE[0]})
#     test_data.append({'text': entry['rejected'], 'label': INVERSE[entry['fallacy_type']]})
import pandas as pd
train_data=downsample_data(pd.DataFrame(df_train), 'train').sample(frac=1, random_state=42).reset_index(drop=True)
test_data=downsample_data(pd.DataFrame(df_test), 'test').sample(frac=1, random_state=42).reset_index(drop=True)
# need to encode the labels

train_data.label = train_data.label.apply(lambda x: CLASSES[x])
test_data.label = test_data.label.apply(lambda x: CLASSES[x])
train_data
df = train_data
downsampled_dfs = []
for label in df['label'].unique():
    # Filter the dataframe by label
    label_df = df[df['label'] == label]
    # Sample the dataframe without replacement
    downsampled_df = label_df.sample(min(len(label_df), 500), random_state=42)
    downsampled_dfs.append(downsampled_df)

# Concatenate all the downsampled dataframes
balanced_df = pd.concat(downsampled_dfs, ignore_index=True)

# Check the new value counts to confirm balancing
train_data = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch

# train_data = pd.DataFrame(train_data)
# train_data = train_data.sample(frac=1).reset_index(drop=True)[:1000]
# test_data = pd.DataFrame(test_data)
# test_data = test_data.sample(frac=1).reset_index(drop=True)
# Load the tokenizer and model
model_name = 'bert-large-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=14, ignore_mismatched_sizes=True)

# Assume `data` is your dataset loaded into a DataFrame with columns "text" and "label"
# Replace this with your actual data loading code

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=64, return_tensors='pt')

# Create a Dataset object

train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Split dataset into training and validation
# train_test_dataset = dataset.train_test_split(test_size=0.1, shuffle=True)
# if train_test_dataset['train'].num_rows == 0 or train_test_dataset['test'].num_rows == 0:
#     raise ValueError("One of the splits ended up being empty. Adjust your train-test split.")

train_test_dataset = DatasetDict({
    'train': train_data,
    'test': test_data
})

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=8,               # number of training epochs
    per_device_train_batch_size=8,    # batch size for training
    per_device_eval_batch_size=16,    # batch size for evaluation
    warmup_steps=0,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.1,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
    logging_steps=10,
    eval_steps=50,
    learning_rate=2e-6,
    save_steps=3000,
    evaluation_strategy="steps",
    save_strategy="steps"
)
import numpy as np
from collections import Counter
def compute_metrics(pred):
    predictions, labels = pred.predictions, pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    
    
    misclassified = []
    for i in range(predictions.shape[0]):
       if predictions[i] != labels[i]:
           misclassified.append(INVERSE[predictions[i]])

    return {
        'accuracy': (predictions == labels).mean(),
        'misclassified': Counter(misclassified)
    }
 
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_dataset['train'],
    eval_dataset=train_test_dataset['test'],
    compute_metrics=compute_metrics # Implement more sophisticated metrics as needed
)

# Train the model
trainer.train()


/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1455 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,Misclassified
50,2.625600,2.574665,0.133333,"Counter({'intentional': 869, 'equivocation': 230, 'faulty generalization': 91, 'fallacy of relevance': 68, 'appeal to emotion': 3})"
100,2.444100,2.328844,0.267354,"Counter({'intentional': 535, 'fallacy of relevance': 284, 'faulty generalization': 93, 'appeal to emotion': 84, 'fallacy of extension': 26, 'false causality': 24, 'equivocation': 12, 'fallacy of credibility': 4, 'ad hominem': 4})"
150,2.271900,2.100634,0.474914,"Counter({'intentional': 258, 'fallacy of relevance': 166, 'appeal to emotion': 92, 'false dilemma': 76, 'false causality': 54, 'circular reasoning': 37, 'faulty generalization': 33, 'fallacy of extension': 24, 'fallacy of credibility': 13, 'ad hominem': 5, 'ad populum': 4, 'fallacy of logic': 1, 'equivocation': 1})"
200,2.030000,1.846657,0.645361,"Counter({'circular reasoning': 96, 'false causality': 91, 'intentional': 91, 'false dilemma': 72, 'fallacy of relevance': 59, 'appeal to emotion': 31, 'fallacy of extension': 25, 'ad populum': 23, 'fallacy of credibility': 8, 'fallacy of logic': 8, 'faulty generalization': 8, 'Not a Fallacy': 3, 'ad hominem': 1})"
250,1.824900,1.617299,0.696907,"Counter({'false causality': 124, 'circular reasoning': 118, 'false dilemma': 34, 'appeal to emotion': 33, 'intentional': 31, 'ad populum': 25, 'fallacy of logic': 23, 'fallacy of extension': 18, 'fallacy of relevance': 18, 'Not a Fallacy': 6, 'fallacy of credibility': 4, 'faulty generalization': 4, 'ad hominem': 3})"
300,1.677900,1.445766,0.736082,"Counter({'circular reasoning': 125, 'false causality': 96, 'fallacy of logic': 41, 'ad populum': 28, 'intentional': 23, 'appeal to emotion': 21, 'fallacy of extension': 16, 'false dilemma': 11, 'Not a Fallacy': 9, 'fallacy of credibility': 7, 'ad hominem': 3, 'faulty generalization': 2, 'fallacy of relevance': 2})"
350,1.511800,1.301854,0.743643,"Counter({'circular reasoning': 115, 'false causality': 94, 'fallacy of logic': 57, 'appeal to emotion': 32, 'ad populum': 20, 'intentional': 17, 'fallacy of extension': 15, 'Not a Fallacy': 10, 'fallacy of credibility': 6, 'fallacy of relevance': 3, 'ad hominem': 2, 'false dilemma': 1, 'faulty generalization': 1})"
400,1.383700,1.188743,0.752577,"Counter({'circular reasoning': 122, 'false causality': 76, 'fallacy of logic': 76, 'appeal to emotion': 20, 'fallacy of extension': 15, 'intentional': 14, 'Not a Fallacy': 13, 'ad populum': 10, 'fallacy of credibility': 9, 'fallacy of relevance': 2, 'ad hominem': 2, 'faulty generalization': 1})"
450,1.313600,1.099768,0.767010,"Counter({'circular reasoning': 103, 'false causality': 73, 'fallacy of logic': 71, 'Not a Fallacy': 22, 'ad populum': 15, 'fallacy of extension': 15, 'appeal to emotion': 14, 'intentional': 13, 'fallacy of credibility': 8, 'fallacy of relevance': 3, 'ad hominem': 1, 'faulty generalization': 1})"
500,1.189500,1.024359,0.770447,"Counter({'circular reasoning': 101, 'false causality': 70, 'fallacy of logic': 65, 'intentional': 22, 'Not a Fallacy': 21, 'fallacy of extension': 16, 'appeal to emotion': 15, 'ad populum': 10, 'fallacy of credibility': 9, 'fallacy of relevance': 2, 'ad hominem': 2, 'faulty generalization': 1})"


Trainer is attempting to log a value of "Counter({'intentional': 869, 'equivocation': 230, 'faulty generalization': 91, 'fallacy of relevance': 68, 'appeal to emotion': 3})" of type <class 'collections.Counter'> for key "eval/misclassified" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "Counter({'intentional': 535, 'fallacy of relevance': 284, 'faulty generalization': 93, 'appeal to emotion': 84, 'fallacy of extension': 26, 'false causality': 24, 'equivocation': 12, 'fallacy of credibility': 4, 'ad hominem': 4})" of type <class 'collections.Counter'> for key "eval/misclassified" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "Counter({'intentional': 258, 'fallacy of relevance': 166, 'appeal to emotion': 92, 'false dilemma': 76, 'false causality': 54, 'circular reasoning': 37, 'faulty 

TrainOutput(global_step=824, training_loss=1.5498591895821026, metrics={'train_runtime': 844.2856, 'train_samples_per_second': 62.074, 'train_steps_per_second': 0.976, 'total_flos': 6105329894627328.0, 'train_loss': 1.5498591895821026, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

In [1]:
trainer.save_model('/root/Logical-Fallacies/models/fallacy/clf')
tokenizer.save_pretrained('/root/Logical-Fallacies/models/fallacy/clf')

NameError: name 'trainer' is not defined

In [4]:
def predict(text):
    inputs = tokenizer(text, return_tensors='pt').to('cuda')
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probs

def predict_fallacy(prompt, argument):
    if prompt: 
        input_text = prompt + " " + argument
    else:
        input_text = argument
    probs = predict(input_text)

    predicted_label = torch.argmax(probs)
    # for key, value in CLASSES.items():
    #     if value == predicted_label:
    #         predicted_label = key
    return predicted_label

In [5]:

keys = ['Not a Fallacy', 'ad hominem', 'ad populum', 'appeal to emotion',
 'circular reasoning', 'equivocation', 'fallacy of relevance',
 'false causality' ,'false dilemma' ,'faulty generalization']

d = {k: i for i, k in enumerate(keys)}
d

{'Not a Fallacy': 0,
 'ad hominem': 1,
 'ad populum': 2,
 'appeal to emotion': 3,
 'circular reasoning': 4,
 'equivocation': 5,
 'fallacy of relevance': 6,
 'false causality': 7,
 'false dilemma': 8,
 'faulty generalization': 9}

In [45]:
df_test = pd.DataFrame(df_test)

chosen = df_test['chosen'].unique()

rejected = df_test['rejected'].unique()
accuracy_chosen = 0 
accuracy_rejected = 0
from tqdm import tqdm
for i, entry in tqdm(df_test.iterrows(), total=len(df_test)):
    instruction = entry['prompt']
    chosen = entry['chosen']
    rejected = entry['rejected']
    pred = predict_fallacy(instruction, chosen)
    if pred == 0:
        accuracy_chosen += 1
    
    pred_rejected = predict_fallacy(instruction, rejected)
 

    if le.inverse_transform([pred_rejected.detach().item()])[0] == INVERSE[entry['fallacy_type']]:
        accuracy_rejected += 1


print(accuracy_chosen)
print(accuracy_rejected)

  0%|          | 0/1319 [00:00<?, ?it/s]

  0%|          | 0/1319 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
model = AutoModelForSequenceClassification.from_pretrained('/root/Logical-Fallacies/models/fallacy/clf').to('cuda')
tokenizer = AutoTokenizer.from_pretrained('/root/Logical-Fallacies/models/fallacy/clf')

def predict(text):
    inputs = tokenizer(text, return_tensors='pt').to('cuda')
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probs

def predict_fallacy(prompt, argument):
    if prompt: 
        input_text = prompt + " " + argument
    else:
        input_text = argument
    probs = predict(input_text)
    
    predicted_label = torch.argmax(probs)

    # for key, value in CLASSES.items():
    #     if value == predicted_label:
    #         predicted_label = key
    return predicted_label
keys = ['Not a Fallacy', 'ad hominem', 'ad populum', 'appeal to emotion',
 'circular reasoning', 'equivocation', 'fallacy of relevance',
 'false causality' ,'false dilemma' ,'faulty generalization']

d = {k: i for i, k in enumerate(keys)}
d

import json 
with open('/root/Logical-Fallacies/results/llama/arguments/sft/f-rate.json', 'r') as f:
    data = json.load(f)
acc= 0
misclassified = []
from tqdm import tqdm
for entry in tqdm(data):
    f_type = entry['fallacy_type']
    if f_type=='None':
        f_type = 'Not a Fallacy'

    argument = entry['argument']
    topic = entry['topic']
    pred = predict_fallacy(None, argument)
    pred= INVERSE[pred.detach().item()]

    if pred != f_type:
        misclassified.append(pred)

    if pred == f_type: 
       acc +=1

100%|██████████| 400/400 [00:12<00:00, 32.38it/s]


In [7]:
from collections import Counter 
Counter(misclassified)

Counter({'Not a Fallacy': 86,
         'ad populum': 1,
         'false causality': 12,
         'circular reasoning': 60,
         'intentional': 13,
         'fallacy of logic': 10,
         'ad hominem': 5,
         'fallacy of relevance': 11,
         'fallacy of credibility': 3})

In [8]:
acc

199

In [56]:
from tqdm import tqdm
print(le.inverse_transform([3]))
print(le.classes_)
num_miss = 0
for i, entry in tqdm(enumerate(df_test)):
    chosen = entry['chosen']
    rejected = entry['rejected']
    chosen_pred = predict_fallacy(None, chosen)
    rejected_pred = predict_fallacy(None, rejected)

    if chosen_pred != 0:
        print(chosen)
        num_miss+=1
        print(chosen_pred)
    # if le.inverse_transform([rejected_pred.detach().cpu().item()])[0] !=INVERSE[entry['fallacy_type']]:
    #     print(rejected)
    #     print(INVERSE[entry['fallacy_type']])
    #     print(le.inverse_transform([rejected_pred.detach().cpu().item()])[0])
    #     num_miss+=1

    if num_miss == 5:
        break



['appeal to emotion']
['Not a Fallacy' 'ad hominem' 'ad populum' 'appeal to emotion'
 'circular reasoning' 'equivocation' 'fallacy of relevance'
 'false causality' 'false dilemma' 'faulty generalization']


14it [00:00, 14.54it/s]

If people feel strongly enough to protest loudly, their opinion should be taken into consideration.
tensor(7, device='cuda:0')


25it [00:01, 14.28it/s]

If journalism was subsidized, Newspapers could keep their level of readership up in the digital age.
tensor(7, device='cuda:0')
If journalism was subsidized, Newspapers could keep their level of readership up in the digital age.
tensor(7, device='cuda:0')
If journalism was subsidized, Newspapers could keep their level of readership up in the digital age.
tensor(7, device='cuda:0')
If journalism was subsidized, Newspapers could keep their level of readership up in the digital age.
tensor(7, device='cuda:0')
